In [1]:
# Download and unzip
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzf cifar-10-python.tar.gz
!mkdir ResNet18_color ResNet18_color/pixel_data

--2022-12-03 09:56:39--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Translacja www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Łączenie się z www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 170498071 (163M) [application/x-gzip]
Zapis do: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162,60M  25,4MB/s    w 7,2s     

2022-12-03 09:56:46 (22,5 MB/s) - zapisano ‘cifar-10-python.tar.gz’ [170498071/170498071]



In [2]:
import os
import glob
import pickle

import numpy as np
from skimage.io import imsave
from PIL import Image

In [3]:
ResNet18_color = "ResNet18_color/"
ResNet18_gray = "ResNet18_gray/"
PIXELS_DIR = f"{ResNet18_color}pixel_data/"
PIXELS_DIR_GRAY = f"{ResNet18_gray}pixel_data_grayscale/"


def unpack_file(fname):
    with open(fname, "rb") as f:
        result = pickle.load(f, encoding="bytes")
    return result


def save_as_image(img_flat, fname, label):
    # consecutive 1024 entries store color channels of 32x32 image
    img_R = img_flat[0:1024].reshape((32, 32))
    img_G = img_flat[1024:2048].reshape((32, 32))
    img_B = img_flat[2048:3072].reshape((32, 32))
    img = np.dstack((img_R, img_G, img_B))

    imsave(os.path.join(PIXELS_DIR + str(label), fname), img)

    
for i in range(10):
    os.makedirs(f"{PIXELS_DIR}{'train/'}{i}", exist_ok=True)
    os.makedirs(f"{PIXELS_DIR}{'test/'}{i}", exist_ok=True)
    os.makedirs(f"{PIXELS_DIR_GRAY}{'train/'}{i}", exist_ok=True)
    os.makedirs(f"{PIXELS_DIR_GRAY}{'test/'}{i}", exist_ok=True)
                 
# use "data_batch_*" for just the training set
for batch in glob.glob("cifar-10-batches-py/*_batch*"):
    print(batch[20:])
    if batch[20:] == "test_batch":
        dataset = "test"
    else:
        dataset = "train"
        
    data = unpack_file(batch)

    for i in range(10000):
        img_flat = data[b"data"][i]
        fname = data[b"filenames"][i].decode()
        label = data[b"labels"][i]

        # save the image and store the label
        save_as_image(img_flat, fname, f"{dataset}/{label}")

test_batch
data_batch_1
data_batch_2
data_batch_4


/tmp/ipykernel_10972/3224075126.py:20: UserWarning: ResNet18_color/pixel_data/train/2/flying_bird_s_001061.png is a low contrast image
  imsave(os.path.join(PIXELS_DIR + str(label), fname), img)


data_batch_3
data_batch_5


In [4]:
for folder in os.listdir(PIXELS_DIR):
    for folder2 in os.listdir(f"{PIXELS_DIR}{folder}"):
        for file in os.listdir(f"{PIXELS_DIR}{folder}/{folder2}"):
            img = Image.open(f"{PIXELS_DIR}{folder}/{folder2}/{file}").convert('L')
            img.save(f"{PIXELS_DIR_GRAY}{folder}/{folder2}/{file}")